In [1]:
import module_name
print(module_name.__file__)


/home/sokhorn/.local/lib/python3.8/site-packages/module_name/__init__.py


In [2]:
import pandas as pd
import numpy as np
# Import all libraries for the rest of the blog post
from scipy.integrate import quad
import scipy.stats as st
import timeit
import math
from mlxtend.frequent_patterns import apriori, association_rules


In [3]:
df = pd.read_csv("/home/sokhorn/sokhorn/dataSet/sample/sampleTestFuzzy.csv")
df


,InvoiceNo,StockCode,Quantity
0,1,A,1
1,1,B,1
2,1,D,1
3,2,A,1
4,2,B,1
5,2,C,1
6,2,D,1
7,3,B,1
8,3,D,1
9,4,B,1


In [4]:
itemset = df.groupby(['InvoiceNo', 'StockCode'])['Quantity'].sum(
).unstack().reset_index().fillna(0).set_index("InvoiceNo").astype(int)
itemset


StockCode,A,B,C,D,E,F
InvoiceNo,,,,,,
1,1,1,0,1,0,0
2,1,1,1,1,0,0
3,0,1,0,1,0,0
4,0,1,1,1,1,0
5,1,0,1,0,1,0
6,0,1,0,1,0,1
7,1,0,0,0,1,1
8,0,0,1,0,0,1
9,0,1,1,0,0,1


In [5]:
# Sebastian Raschka 2014-2020
# myxtend Machine Learning Library Extensions
# Author: Sebastian Raschka <sebastianraschka.com>
#
# License: BSD 3 clause

import numpy as np
import pandas as pd


def generate_new_combinations(old_combinations):
    items_types_in_previous_step = np.unique(old_combinations.flatten())
    for old_combination in old_combinations:
        max_combination = old_combination[-1]  # get a single item in the last
        mask = items_types_in_previous_step > max_combination
        valid_items = items_types_in_previous_step[mask]
        old_tuple = tuple(old_combination)
        for item in valid_items:

            yield from old_tuple
            yield item


def generate_new_combinations_low_memory(old_combinations, X, min_support,
                                         is_sparse):

    items_types_in_previous_step = np.unique(old_combinations.flatten())
    rows_count = X.shape[0]
    threshold = min_support * rows_count
    for old_combination in old_combinations:
        max_combination = old_combination[-1]
        mask = items_types_in_previous_step > max_combination
        valid_items = items_types_in_previous_step[mask]
        old_tuple = tuple(old_combination)
        if is_sparse:
            mask_rows = X[:, old_tuple].toarray().all(axis=1)
            X_cols = X[:, valid_items].toarray()
            supports = X_cols[mask_rows].sum(axis=0)
        else:
            mask_rows = X[:, old_tuple].all(axis=1)
            supports = X[mask_rows][:, valid_items].sum(axis=0)
        valid_indices = (supports >= threshold).nonzero()[0]
        for index in valid_indices:
            yield supports[index]
            yield from old_tuple
            yield valid_items[index]


def apriori(df, min_support=0.5, use_colnames=False, max_len=None, verbose=0,
            low_memory=False, minimumInterest=0.07):

    start = timeit.default_timer()

    def _support(_x, _n_rows, _is_sparse):
        # print(f"X : {_x.astype(int)}")
        # print(f"sum : {np.sum(_x, axis=1)}")
        out = (np.sum(_x, axis=0) / _n_rows)
        return np.array(out).reshape(-1)

    def _min_interest(support_each_item, support, _n_rows):
        return abs(support - np.prod(support_each_item.sum(axis=0) / rows_count, axis=1))

    if min_support < 0.:
        raise ValueError('`min_support` must be a positive '
                         'number within the interval `(0, 1]`. '
                         'Got %s.' % min_support)

    if hasattr(df, "sparse"):
        # DataFrame with SparseArray (pandas >= 0.24)
        if df.size == 0:
            X = df.values
        else:
            X = df.sparse.to_coo().tocsc()
        is_sparse = True
    else:
        # dense DataFrame
        X = df.values
        is_sparse = False

    support = _support(X, X.shape[0], is_sparse)
    ary_col_idx = np.arange(X.shape[1])
    support_dict = {1: support[support >= min_support]}
    itemset_dict = {1: ary_col_idx[support >= min_support].reshape(-1, 1)}
    max_itemset = 1
    rows_count = float(X.shape[0])

    all_ones = np.ones((int(rows_count), 1))

    while max_itemset and max_itemset < (max_len or float('inf')):
        next_max_itemset = max_itemset + 1
        # conver from generator to numpy
        combin = generate_new_combinations(itemset_dict[max_itemset])
        combin = np.fromiter(combin, dtype=int)
        combin = combin.reshape(-1, next_max_itemset)
        # print(combin)
        if combin.size == 0:
            break
        if verbose:
            print(
                '\rProcessing %d combinations | Sampling itemset size %d' %
                (combin.size, next_max_itemset), end="")
        if is_sparse:
            _bools = X[:, combin[:, 0]] == all_ones
            for n in range(1, combin.shape[1]):
                _bools = _bools & (X[:, combin[:, n]] == all_ones)
        else:
            _bools = np.all(X[:, combin], axis=2)

        support = _support(np.array(_bools), rows_count, is_sparse)
        _mask = (support >= min_support).reshape(-1)
        mask_interest = (support == min_support)
        support_each_item = X[:, combin]  # maybe slow here

        s = np.array(support[_mask])
        i = np.array(combin[_mask])
        if any(_mask):
            itemset_dict[next_max_itemset] = i
            support_dict[next_max_itemset] = s

            max_itemset = next_max_itemset
        else:
            break

        # interest = _min_interest(
        #     support_each_item, support, rows_count)
        # min_interest_mask = (interest >= minimumInterest).reshape(-1)

        # if any(mask_interest):
        #     support_equal = np.array(support[mask_interest])
        #     item_equal = np.array(combin[mask_interest])
        #     print(item_equal)

    all_res = []
    for k in sorted(itemset_dict):
        support = pd.Series(support_dict[k])
        itemsets = pd.Series([frozenset(i) for i in itemset_dict[k]],
                             dtype='object')

        res = pd.concat((support, itemsets), axis=1)
        all_res.append(res)

    res_df = pd.concat(all_res)
    res_df.columns = ['support', 'itemsets']
    if use_colnames:
        mapping = {idx: item for idx, item in enumerate(df.columns)}
        res_df['itemsets'] = res_df['itemsets'].apply(lambda x: frozenset([
            mapping[i] for i in x]))
    res_df = res_df.reset_index(drop=True)
    if verbose:
        print()  # adds newline if verbose counter was used

    stop = timeit.default_timer()
    execution_time = stop - start
    # print(f"execution time : {execution_time}")
    return res_df, execution_time


In [84]:
def sampleSize(accurateResult, probabilityRequirement):
    n = 0
    z_value = st.norm.ppf((probabilityRequirement + 1) / 2)
    n = (pow(z_value, 2)) / (4 * pow(accurateResult, 2))
    n = n + 1
    return int(n)


def instanceSelection(sampleSize, m):
    X = set({})
    a = 2
    b = 3
    x0 = 5
    c = x0
    X.update({x0})
    while(len(X) != sampleSize):
        x = (a * c + b) % m
        if not set({x}).issubset(X):
            X.update({x})
        else:
            break
        c = x
    return X


def randomDatabset(itemset, sample_size):
    random_selection = instanceSelection(
        sampleSize=sample_size, m=len(itemset))
    random_selection = np.fromiter(
        random_selection, int, len(random_selection))
    j_index = np.where(random_selection < itemset.index.values.max() - 1,
                       random_selection + 1,  random_selection)
    return itemset.iloc[j_index]


def a_avgsupp(m, k, N):

    n_over_N = sum([math.pow(m / N, i) for i in range(k, m)])

    operation = 1 / (m - k + 1)
    return n_over_N * operation


def a_b_avg(supports):
    return min(supports), max(supports), sum(supports) / len(supports)


def actul_minimum_support(a, b, r_min, n=1):
    return ((pow(b, n) - pow(a, n)) * r_min + pow(a, n))


def lean(supports, avg, m):
    less_avg = np.apply_along_axis(
        lambda x: x < avg, arr=supports[:m], axis=0).sum()
    greater_avg = np.apply_along_axis(
        lambda x: x > avg, arr=supports[:m], axis=0).sum()
    return (less_avg - greater_avg) / m


def aproximate_minsupport(r_min, min_supp, max_min, n=1):
    a_n = pow(min_supp, n)
    b_n = pow(max_min, n)
    x_n = (r_min - (a_n / (a_n - b_n))) * (b_n - a_n)
    return x_n


def actual_min_support(dataset, r_minsupp, m, k=0, n=3):
    a = 1 / len((dataset))
    N = dataset.shape[1]
    itemset, time = apriori(
        df=dataset, min_support=r_minsupp, use_colnames=True)
    b = itemset['support'].max()
    avg = a_avgsupp(N=N, m=m, k=k)
    if a <= avg:
        r_minsupp = (r_minsupp * 2*(math.pow(avg, n) - pow(a, n))
                     ) - pow(a, n) / (2 * (pow(a, n) - pow(avg, n)))
        r_minsupp = pow(r_minsupp, 1 / n)
    elif avg <= b:
        r_minsupp = (r_minsupp * (2*(pow(b, n) - pow(avg, n)))) - \
            (pow(b, n) - 2 * pow(avg, n)) / (2 * (pow(b, n) - pow(avg, n)))
        r_minsupp = pow(r_minsupp, 1 / n)
    return itemset, time,  r_minsupp


m = int(itemset.sum(axis=1).sum() / len(itemset))  # avg att per rows


In [85]:

apriori_fre_itemset, apriori_time, act_min_aproximate = actual_min_support(
    dataset=itemset, r_minsupp=0.3, m=m,)
print(f"Actual min support : {act_min_aproximate}, compute in {apriori_time}s")
apriori_fre_itemset


Actual min support : 0.38187245235702505, compute in 0.012042465999911656s


,support,itemsets
0,0.5,(A)
1,0.7,(B)
2,0.6,(C)
3,0.6,(D)
4,0.3,(E)
5,0.5,(F)
6,0.3,"(A, B)"
7,0.3,"(A, C)"
8,0.3,"(A, D)"
9,0.4,"(B, C)"


In [103]:
def usr_min_and_actual_min_support(dataset, r_minsupp, m, metric="confidence", min_threshold=0.6):
    apriori_fre_itemset, apriori_time, act_min_aproximate = actual_min_support(
        dataset=itemset, r_minsupp=r_minsupp, m=m,)

    print(
        f"User minimum support {r_minsupp} , actual minimum support {act_min_aproximate}")

    act_min_supp_fre_itemset, act_time = apriori(
        df=dataset, min_support=act_min_aproximate, use_colnames=True)

    apriori_rule = []
    act_rule = []
    if(not apriori_fre_itemset.empty):
        apriori_rule = association_rules(
            apriori_fre_itemset, metric=metric, min_threshold=min_threshold)

    if(not act_min_supp_fre_itemset.empty):
        act_rule = association_rules(
            act_min_supp_fre_itemset, metric=metric, min_threshold=min_threshold)
    print(
        f"================\nUser support : {r_minsupp} => actual minimum support : {act_min_aproximate}\nFrequent Itemst Apriori {len(apriori_fre_itemset )} => actual Frequent Itemst base on actual minimum suport : {len(apriori_fre_itemset )}\nRulle Apriori {len(apriori_rule)} => actual rule base on actual minimum support : {len(act_rule)}\n================")
    return {
        "Support": {
            "r_min_support": r_minsupp,
            "act_support": act_min_aproximate
        },
        "Itemset": {
            "apriori_fre_itemset": apriori_fre_itemset,
            "act_min_supp_fre_itemset": act_min_supp_fre_itemset,
        }, "Association Rules": {
            "apriori_rule": apriori_rule,
            "act_rule": act_rule,
        },
        "Time": {
            "apriori_time": apriori_time,
            "act_time": act_time,
        }
    }


algorithms = usr_min_and_actual_min_support(
    dataset=itemset, r_minsupp=0.7000000000000001, m=m,)
print(type(algorithms))
apriori_fre_itemset = algorithms['Itemset']["apriori_fre_itemset"]
act_fre_itemset = algorithms['Itemset']["act_min_supp_fre_itemset"]

rule_apri_fre_itemset = algorithms['Association Rules']["apriori_rule"]
rule_act_fre_itemset = algorithms['Association Rules']["act_rule"]

time_apri_fre_itemset = algorithms['Time']["apriori_time"]
time_act_min_supp_fre_itemset = algorithms['Time']["act_time"]
act_fre_itemset


User minimum support 0.7000000000000001 , actual minimum support 0.49580400217945847
User support : 0.7000000000000001 => actual minimum support : 0.49580400217945847
Frequent Itemst Apriori 0 => actual Frequent Itemst base on actual minimum suport : 0
Rulle Apriori 0 => actual rule base on actual minimum support : 2
<class 'dict'>


,support,itemsets
0,0.5,(A)
1,0.7,(B)
2,0.6,(C)
3,0.6,(D)
4,0.5,(F)
5,0.6,"(B, D)"


In [104]:
len_apri_fre_itemset_result = []
len_act_min_supp_fre_itemset_result = []

time_apri_fre_itemset_resullt = []
time_act_min_supp_fre_itemset_result = []

rule_apri_fre_itemset_result = []
ruule_act_min_supp_fre_itemset_result = []

supports = np.linspace(0.1, 1, 10)

act_supports = []
rminsupp_supports = []


for i in range(len(supports)):
    print(f"{i}/{len(supports)}")
    algorithms = usr_min_and_actual_min_support(
        dataset=itemset, r_minsupp=supports[i], m=m,)

    rminsupp_supports.append(algorithms['Support']["r_min_support"])
    act_supports.append(algorithms['Support']["act_support"])

    apriori_fre_itemset = algorithms['Itemset']["apriori_fre_itemset"]
    act_fre_itemset = algorithms['Itemset']["act_min_supp_fre_itemset"]

    rule_apri_fre_itemset = algorithms['Association Rules']["apriori_rule"]
    rule_act_fre_itemset = algorithms['Association Rules']["act_rule"]

    time_apri_fre_itemset = algorithms['Time']["apriori_time"]
    time_act_min_supp_fre_itemset = algorithms['Time']["act_time"]

    len_apri_fre_itemset_result.append(len(apriori_fre_itemset))
    len_act_min_supp_fre_itemset_result.append(len(act_fre_itemset))

    time_apri_fre_itemset_resullt.append(time_apri_fre_itemset)
    time_act_min_supp_fre_itemset_result.append(time_act_min_supp_fre_itemset)

    rule_apri_fre_itemset_result.append(len(rule_apri_fre_itemset))
    ruule_act_min_supp_fre_itemset_result.append(len(rule_act_fre_itemset))


0/10
User minimum support 0.1 , actual minimum support 0.28269112752790976
User support : 0.1 => actual minimum support : 0.28269112752790976
Frequent Itemst Apriori 43 => actual Frequent Itemst base on actual minimum suport : 43
Rulle Apriori 58 => actual rule base on actual minimum support : 13
1/10
User minimum support 0.2 , actual minimum support 0.3395238518925682
User support : 0.2 => actual minimum support : 0.3395238518925682
Frequent Itemst Apriori 25 => actual Frequent Itemst base on actual minimum suport : 25
Rulle Apriori 32 => actual rule base on actual minimum support : 3
2/10
User minimum support 0.30000000000000004 , actual minimum support 0.3818724523570251
User support : 0.30000000000000004 => actual minimum support : 0.3818724523570251
Frequent Itemst Apriori 7 => actual Frequent Itemst base on actual minimum suport : 7
Rulle Apriori 3 => actual rule base on actual minimum support : 3
3/10
User minimum support 0.4 , actual minimum support 0.4164692619890159
User supp

In [105]:
act_supports

[0.28269112752790976,
 0.3395238518925682,
 0.3818724523570251,
 0.4164692619890159,
 0.44611175721103796,
 0.472265044451383,
 0.49580400217945847,
 0.517297846912023,
 0.5371401519786658,
 0.5556157200726833]

In [114]:
summary = pd.DataFrame(columns=["User support", "Actual support",
                                "Frequent Itemset Apriori", "Actual Frequent Itemset actual minsupport",
                                "Apriori Rule", "Actual Rule", "Apriori Time",  "Actual time"])
for i in range(len(rminsupp_supports)):
    summary.loc[i] = [
        rminsupp_supports[i],
        act_supports[i],
        len_apri_fre_itemset_result[i],
        len_act_min_supp_fre_itemset_result[i],
        rule_apri_fre_itemset_result[i],
        ruule_act_min_supp_fre_itemset_result[i],
        time_apri_fre_itemset_resullt[i],
        time_act_min_supp_fre_itemset_result[i]
    ]
summary


,User support,Actual support,Frequent Itemset Apriori,Actual Frequent Itemset actual minsupport,Apriori Rule,Actual Rule,Apriori Time,Actual time
0,0.1,0.282691,43.0,16.0,58.0,13.0,0.010090,0.004760
1,0.2,0.339524,25.0,7.0,32.0,3.0,0.009543,0.010169
2,0.3,0.381872,7.0,7.0,3.0,3.0,0.006388,0.009999
3,0.4,0.416469,7.0,6.0,3.0,2.0,0.006568,0.004113
4,0.5,0.446112,6.0,6.0,2.0,2.0,0.006893,0.006171
5,0.6,0.472265,4.0,6.0,2.0,2.0,0.003642,0.004961
6,0.7,0.495804,0.0,6.0,0.0,2.0,0.002076,0.003062
7,0.8,0.517298,0.0,4.0,0.0,2.0,0.004350,0.002823
8,0.9,0.537140,0.0,4.0,0.0,2.0,0.001631,0.004282
9,1.0,0.555616,0.0,4.0,0.0,2.0,0.001909,0.002730


In [ ]:
fre_apriori_itemsets, time = apriori(
    itemset, min_support=0.2, use_colnames=True)
fre_apriori_itemsets


,support,itemsets
0,0.5,(A)
1,0.7,(B)
2,0.6,(C)
3,0.6,(D)
4,0.3,(E)
5,0.5,(F)
6,0.3,"(B, A)"
7,0.3,"(C, A)"
8,0.3,"(D, A)"
9,0.2,"(E, A)"


In [ ]:
def f(x):
    return (0.4 * x) - 0.7


f(2)


0.10000000000000009

In [ ]:
def g2(x, g1_of_x, ):
    print(x, g1_of_x)
    return (g1_of_x - f(x)) / ((x - 1) * (x - 2))


In [ ]:
def f2(x):
    return 0.07 * x - 0.49


In [ ]:
def g2(x, g1_of_x, ):
    print(x, g1_of_x)
    return (g1_of_x - f(x)) / ((x - 1) * (x - 2))


In [ ]:
s = fre_apriori_itemsets['support']
g2_arrays = np.array([g2(i, s[i]) for i in s.index])
g2_arrays


0 0.5
1 0.7
2 0.6
3 0.6
4 0.3
5 0.5
6 0.3
7 0.3
8 0.3
9 0.2
10 0.2
11 0.4
12 0.6
13 0.3
14 0.3
15 0.2
16 0.3
17 0.2
18 0.2
19 0.3
20 0.2
21 0.3
22 0.2
23 0.2
24 0.2


/tmp/ipykernel_8159/831402850.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  return (g1_of_x - f(x)) / ((x - 1) * (x - 2))


array([ 0.6       ,         inf,         inf,  0.05      , -0.1       ,
       -0.06666667, -0.07      , -0.06      , -0.05238095, -0.04821429,
       -0.04305556, -0.03666667, -0.03181818, -0.03181818, -0.02948718,
       -0.02802198, -0.02571429, -0.02458333, -0.02316176, -0.02156863,
       -0.02076023, -0.01947368, -0.01880952, -0.01796537, -0.01719368])

In [ ]:
for i in s.index[3:6]:
    print(i)
g2_arrays[3:6]


3
4
5


array([ 0.05      , -0.1       , -0.06666667])

In [ ]:
s


0     0.5
1     0.7
2     0.6
3     0.6
4     0.3
5     0.5
6     0.3
7     0.3
8     0.3
9     0.2
10    0.2
11    0.4
12    0.6
13    0.3
14    0.3
15    0.2
16    0.3
17    0.2
18    0.2
19    0.3
20    0.2
21    0.3
22    0.2
23    0.2
24    0.2
Name: support, dtype: float64

In [ ]:
x = list(s.index)
support_dic = {}
[support_dic.update({i: s.values[i]}) for i in x]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
support_dic


{0: 0.5,
 1: 0.7,
 2: 0.6,
 3: 0.6,
 4: 0.3,
 5: 0.5,
 6: 0.3,
 7: 0.3,
 8: 0.3,
 9: 0.2,
 10: 0.2,
 11: 0.4,
 12: 0.6,
 13: 0.3,
 14: 0.3,
 15: 0.2,
 16: 0.3,
 17: 0.2,
 18: 0.2,
 19: 0.3,
 20: 0.2,
 21: 0.3,
 22: 0.2,
 23: 0.2,
 24: 0.2}

In [ ]:
def gk_value(support_dic, x):
    return list(support_dic.values())[x]


def x_k(support_dic,  x):
    return list(support_dic.keys())[x]


In [ ]:
def fk(x_key, support_dic):
    return ((x_key - x_k(support_dic, 2 * x_key - 1)) * gk_value(support_dic, 2 * x_key - 1)) / (x_k(support_dic, 2 * x_key - 1) - x_k(support_dic, 2 * x_key)) + \
        (x_key - x_k(support_dic, 2 * x_key - 1) * gk_value(support_dic, 2 * x_key)
         ) / (x_k(support_dic, 2 * x_key) - x_k(support_dic, 2 * x_key - 1))


fk(2, support_dic=support_dic)


1.7000000000000002

In [ ]:
size = sampleSize(accurateResult=0.01, probabilityRequirement=0.99)
random_selection = instanceSelection(sampleSize=size, m=len(itemset))

random_selected = np.fromiter(random_selection, int, len(random_selection))
RD = itemset.iloc[random_selected]
RD


StockCode,A,B,C,D,E,F
InvoiceNo,,,,,,
10,1,1,1,1,0,1
4,0,1,1,1,1,0
6,0,1,0,1,0,1
2,1,1,1,1,0,0


In [ ]:
a, t = apriori(RD, min_support=0.06, use_colnames=True)


In [ ]:
fre_apriori_itemsets['support'].sum() / len(fre_apriori_itemsets)


0.33599999999999997

In [ ]:
# def lean(supports, avg, m):
#     supports = np.array(supports, dtype=int)
#     less = np.array(supports[:m] < avg, dtype=int).sum()
#     greater = np.array(supports[:m] > avg, dtype=int).sum()
#     return (less - greater) / m

def leans(supports, m):
    size = len(supports)
    avg = supports.sum() / size
    less = np.array(supports < avg, dtype=int).sum()
    greater = np.array(supports > avg, dtype=int).sum()
    return (less - greater) / m


i = itemset
m = int(i.sum(axis=1).sum() / len(i))
fre_apriori_itemsets, time = apriori(
    i, min_support=0.3, use_colnames=True)

leans(supports=fre_apriori_itemsets['support'], m=m)


1.3333333333333333

In [ ]:
f_fsupport = ['VS', 'S', 'SS', 'M',
              'SL', 'L', 'VL']
f_lean = {-1: "LG", 0: "S", 1: "RG"}
f_group = [
    "VL", "L", "L", "SL", "M", "SH", "H", "VH"
]
fuzzy_rule_df = pd.DataFrame(columns=f_fsupport, index=list(f_lean.values()))
fuzzy_rule_df["VS"] = "VL"
fuzzy_rule_df.loc['LG']['S'] = "M"
fuzzy_rule_df.loc['LG']['SS'] = "SH"
fuzzy_rule_df.loc['LG']['M'] = "H"
fuzzy_rule_df.loc['LG']['SL'] = "VH"
fuzzy_rule_df.loc['LG']['L'] = "VH"
fuzzy_rule_df.loc['LG']['VL'] = "VH"

fuzzy_rule_df.loc['S']['S'] = "SL"
fuzzy_rule_df.loc['S']['SS'] = "M"
fuzzy_rule_df.loc['S']['M'] = "SH"
fuzzy_rule_df.loc['S']['SL'] = "H"
fuzzy_rule_df.loc['S']['L'] = "VH"
fuzzy_rule_df.loc['S']['VL'] = "VH"

fuzzy_rule_df.loc['RG']['S'] = "VL"
fuzzy_rule_df.loc['RG']['SS'] = "L"
fuzzy_rule_df.loc['RG']['M'] = "SL"
fuzzy_rule_df.loc['RG']['SL'] = "M"
fuzzy_rule_df.loc['RG']['L'] = "SH"
fuzzy_rule_df.loc['RG']['VL'] = "VH"

fuzzy_rule_df


,VS,S,SS,M,SL,L,VL
LG,VL,M,SH,H,VH,VH,VH
S,VL,SL,M,SH,H,VH,VH
RG,VL,VL,L,SL,M,SH,VH


In [ ]:
a = [1, 2, 5, 4]
tuple(a)


(1, 2, 5, 4)

In [ ]:
def membership_function(x, af=0.23721, cf=0.39147):
    if x < af:
        return 0
    elif (x < cf and x > af):
        return (x - af) / (cf - af)
    elif x > cf:
        return 1


In [ ]:
def fuzzy_frequent_itemset(dataset, f_support, k=0, r_minsupport=0.3, fuzzy_rule_df=fuzzy_rule_df):
    size = sampleSize(accurateResult=0.01, probabilityRequirement=0.99)
    random_selection = instanceSelection(sampleSize=size, m=len(itemset))

    SD = np.fromiter(
        random_selection, int, len(random_selection))

    N = dataset.shape[1]
    fre_apriori_itemsets, time = apriori(
        i, min_support=r_minsupport, use_colnames=True)
    m = int(dataset.sum(axis=1).sum() / len(dataset))  # avg att per rows
    lean = leans(supports=fre_apriori_itemsets['support'], m=m)

    if(lean < 0):
        lean_value = f_lean[-1]
    elif lean == 0:
        lean_value = f_lean[0]
    elif lean > 0:
        lean_value = f_lean[1]

    f_group = fuzzy_rule_df.loc[lean_value][f_support]

    a = itemset.shape[0]
    supports_1_itemset = itemset.sum(axis=1).values / a
    b = max(supports_1_itemset)
    avg = a_avgsupp(N=N, m=m, k=k)


fuzzy_frequent_itemset(dataset=itemset, f_support='VS', r_minsupport=0.3)


In [ ]:

support_count = np.array([0.5, 0.7, 0.6, 0.6, 0.3, 0.5, 0.3, 0.3,
                          0.3, 0.4, 0.6, 0.3, 0.3, 0.3, 0.3, 0.3]) * len(itemset)
support_count


array([5., 7., 6., 6., 3., 5., 3., 3., 3., 4., 6., 3., 3., 3., 3., 3.])

In [ ]:
itemset.head(1)


StockCode,A,B,C,D,E,F
InvoiceNo,,,,,,
1,1,1,0,1,0,0


In [ ]:
def membership(x):
    return (50000 / 7713) * x - (7907 / 5142)


In [ ]:
print(membership_function(x=0.3))


0.40704006223259426
